In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OrdinalEncoder 
import matplotlib.pyplot as plt
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
import plotly.express as px
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
root = '/kaggle/input/house-prices-advanced-regression-techniques/'

# Any results you write to the current directory are saved as output.

In [ ]:
df_price = pd.read_csv(
    os.path.join(root, 'train.csv'))   
price_df = df_price.copy()
price_df.shape

In [ ]:
df_price['SalePrice'].describe()

***Cleaning Data***

In [ ]:
#Finding NaN values
price_df.info()

In [ ]:
#Dropping irrelavant columns and NaN values
price_df.drop(columns = ['LotFrontage', 'Alley', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                         'BsmtFinType1','FireplaceQu', 'BsmtFinType2',  'GarageType', 'GarageYrBlt', 'GarageFinish', 
                         'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'], inplace= True)

#Removing the NaN of the Electrical from the dataset
price_df.drop(price_df[price_df['Electrical'].isnull()].index, inplace= True)



Finding correlations between  feature and result

In [ ]:
import seaborn as sns
corr = price_df.corr(method = 'pearson')
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corr, vmax=1, square=True);
corr = corr.loc['SalePrice':, :].T
corr = corr[(corr['SalePrice'] >0.5) | (corr['SalePrice'] < -0.5)]
corr

Cleaning the data, data with outliers are removed after analyzing the variable plot with the salesprice.

In [ ]:
#Comparing a variable with the SalePrice
price_df.drop(price_df[price_df['GrLivArea'] > 4000].index, inplace = True)
price_df.drop(price_df[price_df['TotalBsmtSF'] > 3000].index, inplace = True)
price_df.drop(price_df[price_df['1stFlrSF'] > 2800].index, inplace = True)

'''fig = make_subplots(rows=5, cols=2, start_cell="top-left")

for i, var in enumerate(corr.index):
    if not var == 'SalePrice':
        i = i+1
        m = 1 if i%2==1 else 2
        k = int(i/2) + 2 - m
        fig.add_trace(go.Scatter(x = price_df[var].values, y = price_df['SalePrice'].values, mode = 'markers'), row=k, col=m)
        fig.update_xaxes(title_text= var, row=k, col=m)
fig.update_layout(height=1200, width=800, title_text="Correlation with SalesPrice")
fig.show()'''

Dealing with Categorical data

In [ ]:
#Distinguishing numerical and categorical data
from pandas.api.types import is_numeric_dtype
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

num = []
categ = []
for row in price_df.columns: 
    numeric = is_numeric_dtype(price_df[row])
    if row not in num and numeric == False:
        categ.append(row)
    elif row not in categ and numeric == True:
        num.append(row)
    
#Identifying non-correlated categorical data

'''for row in categ:
    ce_ord = ce.OrdinalEncoder(cols=[row])
    price_df[row] = ce_ord.fit_transform(price_df[row], price_df['SalePrice'])
for row in categ:
    var = row
    plt.figure()
    plt.scatter(price_df[var],price_df['SalePrice'])
    plt.xlabel(var), plt.ylabel('SalePrice')
corr_all = price_df.corr(method = 'pearson')
corr_all = corr_all.loc['SalePrice':, :].T
corr_all = corr_all[(corr_all['SalePrice'] >0.5) | (corr_all['SalePrice'] < -0.5)]
corr_all

corr1 = corr_all.merge(corr, left_index = True, right_index = True, how= 'outer')
corr1'''

#no categorical data is strongly correlated to the SalePrice


***linear and polynomial regression***

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

train_df = pd.DataFrame(price_df, columns = corr.index)
SalePrice = train_df.pop('SalePrice')
X_train, X_test, y_train, y_test = train_test_split(train_df, SalePrice, random_state=0)


model = LinearRegression()
model.fit(X_train,y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print('R2 value for linear regression train case {}'.format(r2_score(y_train, y_pred_train)))
print('R2 value for linear regression test case {}'.format(r2_score(y_test, y_pred_test)))
print('MSE value for linear regression test case {}'.format(mean_absolute_error(y_test, y_pred_test)))

model_poly = LinearRegression()
polynomial_features= PolynomialFeatures(degree=2)
x_poly_train = polynomial_features.fit_transform(X_train)
x_poly_test = polynomial_features.fit_transform(X_test)
model_poly.fit(x_poly_train,y_train)
y_pred_train = model_poly.predict(x_poly_train)
y_pred_test = model_poly.predict(x_poly_test)
print('R2 value for polynomial regression train case {}'.format(r2_score(y_train, y_pred_train)))
print('R2 value for polynomial regression test case {}'.format(r2_score(y_test, y_pred_test)))
print('MSE value for polynomial regression test case {}'.format(mean_absolute_error(y_test, y_pred_test)))

***Linear regression with Regularization***

In [ ]:
#Using regularization to reduce the overfitiing
# define model
model = Ridge(alpha=20.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=cv, n_jobs=1)
# force scores to be positive
scores = absolute(scores)
print('Mean R2: {} and {}'.format(mean(scores), std(scores)))

In [ ]:
#fiting the model on the training data
model.fit(X_train,y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print('R2 value for linear regression train case {}'.format(r2_score(y_train, y_pred_train)))
print('R2 value for linear regression test case {}'.format(r2_score(y_test, y_pred_test)))
print('MSE value for linear regression test case {}'.format(mean_absolute_error(y_test, y_pred_test)))

***Decision Tree Regressor***

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

# Define model
melbourne_model = DecisionTreeRegressor(min_samples_split = 20, criterion = 'mse')

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# evaluate model
scores = cross_val_score(melbourne_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=1)

# force scores to be positive
scores = absolute(scores)
print('Mean MSE: {} and {}'.format(mean(scores), std(scores)))


In [ ]:
#fiting the model on the training data
melbourne_model.fit(X_train,y_train)
y_pred_train = melbourne_model.predict(X_train)
y_pred_test = melbourne_model.predict(X_test)

print('R2 value for Decision Tree Regressor train case {}'.format(r2_score(y_train, y_pred_train)))
print('R2 value for Decision Tree Regressor test case {}'.format(r2_score(y_test, y_pred_test)))
print('MSE value for Decision Tree Regressor test case {}'.format(mean_absolute_error(y_test, y_pred_test)))